In [1]:
### Download Models

In [20]:
MODELS = {
    'autoregressive.pth': 'https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/autoregressive.pth',
    'clvp2.pth': 'https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/clvp2.pth',
    'diffusion_decoder.pth': 'https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/diffusion_decoder.pth',
    'vocoder.pth': 'https://huggingface.co/jbetker/tortoise-tts-v2/resolve/main/.models/vocoder.pth',
}

In [21]:
MODELS.keys()

dict_keys(['autoregressive.pth', 'clvp2.pth', 'diffusion_decoder.pth', 'vocoder.pth'])

In [23]:
from huggingface_hub import hf_hub_download

models_dir = "model"

for model in MODELS.keys():
    model_path = hf_hub_download(repo_id="Manmay/tortoise-tts", filename=model, cache_dir=models_dir, local_dir_use_symlinks=False)

autoregressive.pth:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

clvp2.pth:   0%|          | 0.00/976M [00:00<?, ?B/s]

diffusion_decoder.pth:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

vocoder.pth:   0%|          | 0.00/391M [00:00<?, ?B/s]

In [24]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [28]:
!cd model &&tar --exclude='models--Manmay--tortoise-tts/blobs/*' -h -cvzf ../model.tar.gz .

./
./.locks/
./.locks/models--Manmay--tortoise-tts/
./.locks/models--Manmay--tortoise-tts/9c6651b9996df6cef6a1fc459738ae207ab60f902ec49b4d0623ca8ab6110d51.lock
./.locks/models--Manmay--tortoise-tts/6097e708cf692eb93bd770880660953935e87e8995eb864819bbe51b7d91342c.lock
./.locks/models--Manmay--tortoise-tts/ea776fc354eabb70cfae145777153483fad72e3e0c5ea345505ded2231a90ce1.lock
./.locks/models--Manmay--tortoise-tts/16e8153e9f8ffb00b116f7f67833df2802fcf81e6bc173acc3b3b4bf9f04189d.lock
./models--Manmay--tortoise-tts/
./models--Manmay--tortoise-tts/blobs/
./models--Manmay--tortoise-tts/snapshots/
./models--Manmay--tortoise-tts/snapshots/50672670cecf2265aa61edb4eef5d1a293a8a373/
./models--Manmay--tortoise-tts/snapshots/50672670cecf2265aa61edb4eef5d1a293a8a373/autoregressive.pth
./models--Manmay--tortoise-tts/snapshots/50672670cecf2265aa61edb4eef5d1a293a8a373/clvp2.pth
./models--Manmay--tortoise-tts/snapshots/50672670cecf2265aa61edb4eef5d1a293a8a373/diffusion_decoder.pth
./models--Manmay--tortoi

In [29]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket_name = sess.default_bucket() # Set a default S3 bucket

prefix = "tortoise-tts"

s3 = boto3.client('s3')

# Specify the local file path
local_file_path = 'model.tar.gz'

s3_key = f"{prefix}/model/model.tar.gz"  # Example: 'backups/file.tar.gz'

# Upload the file to S3
try:
    s3.upload_file(local_file_path, bucket_name, s3_key)
    print(f'File uploaded successfully to s3://{bucket_name}/{s3_key}')
except Exception as e:
    print(f'Error uploading file: {e}')

model_s3 = f"s3://{bucket_name}/{s3_key}"



File uploaded successfully to s3://sagemaker-us-east-1-376678947624/tortoise-tts/model/model.tar.gz


In [30]:
%store bucket_name
%store model_s3
%store prefix

Stored 'bucket_name' (str)
Stored 'model_s3' (str)
Stored 'prefix' (str)


In [41]:
from sagemaker.jumpstart.utils import get_jumpstart_content_bucket

filename = "sample1.wav"
output = "samples/swami-interview.wav"
s3_bucket = get_jumpstart_content_bucket()
s3_prefix = "training-datasets/asr_notebook_data"
s3_client = boto3.client("s3")
s3_client.download_file(s3_bucket, f"{s3_prefix}/{filename}", output)